In [1]:
import os, glob
import pandas as pd
import numpy as np

In [2]:
#path where the psychopy csv files are
subid = "7568"
session = "ses-01"

subject="sub09"
save_prefix=subject+"_ses01"
root=f'/media/pfaffenrot/Elements/postdoc/projects/hippocampus_VASO/derivatives/pipeline/{subid}/{session}/func/'

#path where you want to save the onset files
save_root=f'/media/pfaffenrot/Elements/postdoc/projects/hippocampus_VASO/derivatives/pipeline/{subid}/{session}/func/'


##this will automatically retrieve the raw psychopy files of each run in the "root" folder, provided that 
#you saved these files with run number "run1", "run2" and "run3" in their names
runs=[glob.glob(root+"/"+subject+"*run1*csv")[0],
     glob.glob(root+"/"+subject+"*run2*csv")[0],
     glob.glob(root+"/"+subject+"*run3*csv")[0]]

In [3]:
#argument run is a list of the path of each run, argument save_root is where you want to save the files,
#argument save_prefix is subjct number + other info
def get_onset_files(runs,save_root,save_prefix):
    
    for runnumber, run in enumerate(runs):
        df=pd.read_csv(run)
        runnumber_correct=runnumber+1


        #fetch the timing of the first trigger
        keyresp=df["key_resp_waitscan.started"]
        keyresprt=df["key_resp_waitscan.rt"]

        first_TR=keyresp+keyresprt
        first_TR=first_TR[3]

        #substract timing of the first trigger to timings of the events
        event_time=df["text_stim.started"]
        event_onset_corrected=event_time-first_TR

        
        ##calculate onset of button press, and duration of before/after button press for each trial
        before_buttonpress_duration=df["key_resp_find.rt"]
        after_buttonpress_onset_corrected=event_onset_corrected+before_buttonpress_duration
        after_buttonpress_duration=17.6-df["key_resp_find.rt"].astype(float)
        
        #create new df with the onset values
        newdf=pd.DataFrame()
        newdf["condition"]=df["condition"]
        newdf['onset_start_of_trial']=event_onset_corrected
        newdf['onset_afterbuttonpress']=after_buttonpress_onset_corrected
        newdf["duration_beforebuttonpress"]=before_buttonpress_duration
        newdf['duration_afterbuttonpress']=after_buttonpress_duration
        newdf["duration_wholetrial"]=before_buttonpress_duration+after_buttonpress_duration
        
        
        newdf=newdf.iloc[7:,:]
        
        newdf.to_csv(save_root+"/"+save_prefix+"_run"+str(runnumber_correct)+"_onset.csv")


In [4]:
get_onset_files(runs,save_root,save_prefix)